In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers datasets accelerate scikit-learn pandas torch evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd

train_data = pd.read_parquet('/content/drive/MyDrive/train.parquet')
#print(train_data)
test_data = pd.read_parquet('/content/drive/MyDrive/test.parquet')
#print(test_data)
validation_data = pd.read_parquet('/content/drive/MyDrive/validation.parquet')
#print(validation_data)

In [ ]:
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(train_data)
test = Dataset.from_pandas(test_data)
validation = Dataset.from_pandas(validation_data)

data = DatasetDict()
data['train'] = train
data['test'] = test
data['validation'] = validation

print(data)

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
})


In [ ]:
data['train'].features['labels']
print(data['train'].features['labels'])

label_names = ['불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰']

List(Value('int64'))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate
import torch
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report

n_emo = len(label_names)

model_checkpoint = "klue/roberta-large" #한국어 전문 모델(?)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = data.map(tokenize_function, batched=True)

def preprocess_labels(examples):
  processed_batch_label = []
  for label in examples['labels']:
    labels_one_hot_enc = [0.0] * n_emo
    for l in label:
      labels_one_hot_enc[l] = 1.0
    processed_batch_label.append(labels_one_hot_enc)
  return {"labels": processed_batch_label}

tokenized_datasets = tokenized_datasets.map(preprocess_labels, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["ID", "text", "token_type_ids"])
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {"labels": [torch.tensor(x, dtype=torch.float32) for x in examples["labels"]]},
    batched=True
)

tokenized_datasets.set_format("torch")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

...

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets)
print(tokenized_datasets['train'].to_pandas())
print(tokenized_datasets['test'].to_pandas())
print(tokenized_datasets['validation'].to_pandas())

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})
                                                  labels  \
0      [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...   
1      [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...   
2      [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...   
4      [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, ...   
...                                                  ...   
39995  [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, ...   
39996  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
39997  [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, ...   
39998  [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=n_emo, problem_type = "multi_label_classfication")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def eval_metrics(pred):
  logits, labels = pred
  predictions = (torch.sigmoid(torch.from_numpy(logits)).numpy() > 0.5).astype(int)
  labels_int = labels.astype(int) if isinstance(labels, np.ndarray) else np.array(labels).astype(int)
  f1 = f1_score(labels_int, predictions, average='weighted', zero_division = 0)
  return {"f1 score": f1}

In [ ]:
train_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/EmoModel/result",
    overwrite_output_dir = True,
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 2,
    per_device_eval_batch_size = 16,
    fp16 = True,
    num_train_epochs = 6,
    weight_decay = 0.01,
    warmup_steps = 500,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    save_total_limit=1, #용량을 위해 가장 최근 체크포인트만 저장
    metric_for_best_model = "f1 score", #f1이 가장 높은 모델을 저장
    greater_is_better = True,
    logging_dir = "/content/drive/MyDrive/EmoModel/log",
    logging_steps = 500,
    report_to = 'none',
)

In [ ]:
class CustomTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
    labels = inputs.pop("labels")
    labels = labels.to(torch.float32).requires_grad_(False)
    outputs = model(**inputs)
    logits = outputs.get("logits")
    loss_fct = torch.nn.BCEWithLogitsLoss()
    loss = loss_fct(logits, labels)
    return(loss, outputs) if return_outputs else loss

In [ ]:
from transformers import EarlyStoppingCallback

trainer = CustomTrainer(
    model = model,
    args = train_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = eval_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/tmp/ipython-input-3574413647.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 score
1,0.299600,0.289594,0.547434
2,0.272400,0.280776,0.584866
3,0.243800,0.279086,0.603458
4,0.220900,0.284041,0.620530
5,0.199600,0.289337,0.622080
6,0.184300,0.292688,0.620847


TrainOutput(global_step=15000, training_loss=0.24479383646647135, metrics={'train_runtime': 5268.8776, 'train_samples_per_second': 45.55, 'train_steps_per_second': 2.847, 'total_flos': 3.3898469117975616e+16, 'train_loss': 0.24479383646647135, 'epoch': 6.0})

In [ ]:
results = trainer.evaluate(tokenized_datasets['test'])

In [ ]:
print(results)

{'eval_loss': 0.291616827249527, 'eval_f1 score': 0.621115693018265, 'eval_runtime': 21.2625, 'eval_samples_per_second': 235.156, 'eval_steps_per_second': 14.721, 'epoch': 6.0}


In [ ]:
test_pred = trainer.predict(tokenized_datasets['test'])

In [ ]:
print(test_pred)

PredictionOutput(predictions=array([[-4.2460938 , -0.26342773, -0.29858398, ..., -3.6699219 ,
        -3.3789062 , -2.984375  ],
       [-1.8417969 , -0.7270508 ,  2.0722656 , ..., -2.9804688 ,
         1.1572266 , -0.93115234],
       [-0.91015625, -0.24951172, -2.6386719 , ..., -2.5800781 ,
        -2.953125  , -2.4980469 ],
       ...,
       [-4.765625  , -1.4785156 , -0.7006836 , ..., -5.1914062 ,
        -0.19091797, -4.984375  ],
       [-4.90625   ,  3.765625  ,  3.1757812 , ..., -4.421875  ,
        -0.3383789 ,  1.8349609 ],
       [-0.5625    , -1.8095703 , -3.09375   , ..., -1.65625   ,
        -3.3789062 , -0.50878906]], dtype=float32), label_ids=array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       ...,
       [0, 1, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       [0, 1, 0, ..., 0, 0, 1]]), metrics={'test_loss': 0.291616827249527, 'test_f1 score': 0.621115693018265, 'test_runtime': 21.0741, 'test_samples_per_second

In [ ]:
emo_names = ['불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰']

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

path = "/content/drive/MyDrive/EmoModel/result/checkpoint-12500"
load_tokenizer = AutoTokenizer.from_pretrained(path)
load_model = AutoModelForSequenceClassification.from_pretrained(path)

if torch.cuda.is_available():
  load_model.to("cuda")

load_model.eval()
print("모델 로드/초기화 완료")

모델 로드/초기화 완료


In [ ]:
def get_top_emotions(text, tokenizer, model, emo_names, top_n = 5, threshold=0.15):
  input = tokenizer(text, return_tensors = 'pt', truncation = True, padding = True)
  if torch.cuda.is_available():
    input = {k: v.to("cuda") for k, v in input.items()}

  with torch.no_grad():
    output = model(**input)

  logits = output.logits.cpu().numpy()[0]

  probabilities = 1/(1+np.exp(-logits))

  filtered_emo = []
  for i, prob in enumerate(probabilities):
    if prob >= threshold:
      filtered_emo.append((emo_names[i], prob))

  filtered_emo.sort(key=lambda x: x[1], reverse = True)

  top_emo_percent = [
      (name, f"{prob*100: .2f}%")
      for name, prob in filtered_emo[:top_n]
  ]

  return top_emo_percent

In [ ]:
text = "나 너무 힘들어"
top_emo = get_top_emotions(text, load_tokenizer, load_model, emo_names)

print(f"\n텍스트: \"{text}\"")
if top_emo:
    for emotion, percent in top_emo:
        print(f"  - {emotion}: {percent}")
else:
    print("  예측된 감정이 없습니다. (모든 감정 확률이 임계값 미만)")



텍스트: "나 너무 힘들어"
  - 힘듦/지침:  91.10%
  - 슬픔:  85.38%
  - 절망:  66.78%
  - 짜증:  47.88%
  - 서러움:  46.34%


In [ ]:
import pandas as pd

review_path = "/content/drive/MyDrive/all_reviews.csv"

df = pd.read_csv(review_path)

print(df.head())

   Unnamed: 0                  책이름   저자        책id   별점  \
0           1  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
1           2  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
2           3  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
3           4  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
4           5  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   

                                                  리뷰 카테고리  
0  -출판사로부터 책을 제공받아 주관적으로 작성한 글입니다-다수의 잠재적 독자인 비전공...   IT  
1  시대가 바뀌고 있습니다. 이제 AI의 시대 누구와도 누군가와도 협력해야할 시대가 온...   IT  
2  이제는 챗GPT의 시대이다?23년부터 시작된 AI시대는 아직 끝날 기미가 보이지 않...   IT  
3  [출판사로부터 도서를 제공받아 작성한 주관적인 리뷰입니다.]챗GPT는 2022년 1...   IT  
4  [출판사로부터 도서를 제공받아 작성한 주관적인 리뷰입니다.]챗GPT로 대표되는 생성...   IT  


In [ ]:
analyzed_emo = []

count = 0
total_reviews = len(df)

print("Start analyzing")

for index, row in df.iterrows():
  review_text = str(row['리뷰'])

  #print(f"\n--- 현재 분석 중인 리뷰 ({index+1}/{total_reviews}) ---")
  #print(f"원본 리뷰 텍스트: {review_text[:100]}...")

  if not review_text or review_text.strip() == "":
    analyzed_emo.append("None")
  else:
    top_emo = get_top_emotions(review_text, load_tokenizer, load_model, emo_names, top_n = 3, threshold=0.15)

    if top_emo:
      formatted_emo = [f"{name}({percent})" for name, percent in top_emo]
      analyzed_emo.append(", ".join(formatted_emo))
    else:
      analyzed_emo.append("None")

  count += 1

  if count %1000 == 0:
    print(f"진행 상황: {count}/{total_reviews} 완료")

print(f"\n🚨🚨🚨 감정 분석 완료! 총 {count}/{total_reviews}개 리뷰 처리 🚨🚨🚨")

df['emotions'] = analyzed_emo


Start analyzing
진행 상황: 1000/49417 완료
진행 상황: 2000/49417 완료
진행 상황: 3000/49417 완료
진행 상황: 4000/49417 완료
진행 상황: 5000/49417 완료
진행 상황: 6000/49417 완료
진행 상황: 7000/49417 완료
진행 상황: 8000/49417 완료
진행 상황: 9000/49417 완료
진행 상황: 10000/49417 완료
진행 상황: 11000/49417 완료
진행 상황: 12000/49417 완료
진행 상황: 13000/49417 완료
진행 상황: 14000/49417 완료
진행 상황: 15000/49417 완료
진행 상황: 16000/49417 완료
진행 상황: 17000/49417 완료
진행 상황: 18000/49417 완료
진행 상황: 19000/49417 완료
진행 상황: 20000/49417 완료
진행 상황: 21000/49417 완료
진행 상황: 22000/49417 완료
진행 상황: 23000/49417 완료
진행 상황: 24000/49417 완료
진행 상황: 25000/49417 완료
진행 상황: 26000/49417 완료
진행 상황: 27000/49417 완료
진행 상황: 28000/49417 완료
진행 상황: 29000/49417 완료
진행 상황: 30000/49417 완료
진행 상황: 31000/49417 완료
진행 상황: 32000/49417 완료
진행 상황: 33000/49417 완료
진행 상황: 34000/49417 완료
진행 상황: 35000/49417 완료
진행 상황: 36000/49417 완료
진행 상황: 37000/49417 완료
진행 상황: 38000/49417 완료
진행 상황: 39000/49417 완료
진행 상황: 40000/49417 완료
진행 상황: 41000/49417 완료
진행 상황: 42000/49417 완료
진행 상황: 43000/49417 완료
진행 상황: 44000/49417 완료
진행 상황: 45000/49417 완료
진행 

In [ ]:
print("\n감정 분석 결과가 추가된 데이터프레임:")
print(df.head())


감정 분석 결과가 추가된 데이터프레임:
   Unnamed: 0                  책이름   저자        책id   별점  \
0           1  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
1           2  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
2           3  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
3           4  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
4           5  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   

                                                  리뷰 카테고리  \
0  -출판사로부터 책을 제공받아 주관적으로 작성한 글입니다-다수의 잠재적 독자인 비전공...   IT   
1  시대가 바뀌고 있습니다. 이제 AI의 시대 누구와도 누군가와도 협력해야할 시대가 온...   IT   
2  이제는 챗GPT의 시대이다?23년부터 시작된 AI시대는 아직 끝날 기미가 보이지 않...   IT   
3  [출판사로부터 도서를 제공받아 작성한 주관적인 리뷰입니다.]챗GPT는 2022년 1...   IT   
4  [출판사로부터 도서를 제공받아 작성한 주관적인 리뷰입니다.]챗GPT로 대표되는 생성...   IT   

                                       emotions  
0     감동/감탄( 98.61%), 깨달음( 96.89%), 존경( 92.00%)  
1     기대감( 98.93%), 감동/감탄( 96.35%), 존경( 92.90%)  
2   깨달음( 98.19%), 신기함/관심( 96.36%), 기대감( 95.93%)  
3  감동/감탄( 99.17%), 깨달음( 92.92%), 안심/신뢰( 89.30%)  
4   기

In [ ]:
print(df)

       Unnamed: 0                  책이름   저자        책id   별점  \
0               1  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
1               2  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
2               3  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
3               4  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
4               5  비전공자도 이해할 수 있는 챗GPT  박상길  146290174  9.8   
...           ...                  ...  ...        ...  ...   
49412       49413              두고 온 여름  성해나  117873525  9.4   
49413       49414              두고 온 여름  성해나  117873525  9.4   
49414       49415              두고 온 여름  성해나  117873525  9.4   
49415       49416              두고 온 여름  성해나  117873525  9.4   
49416       49417              두고 온 여름  성해나  117873525  9.4   

                                                      리뷰     카테고리  \
0      -출판사로부터 책을 제공받아 주관적으로 작성한 글입니다-다수의 잠재적 독자인 비전공...       IT   
1      시대가 바뀌고 있습니다. 이제 AI의 시대 누구와도 누군가와도 협력해야할 시대가 온...       IT   
2      이제는 챗GPT의 시대이다?23년부터 시작된 AI시대

In [ ]:
output_csv_path = '/content/drive/MyDrive/EmoModel/all_reviews_with_emotions.csv' # <--- 저장할 파일 이름 지정!

df.to_csv(output_csv_path, index=False)

print(f"\n감정 분석 결과가 '{output_csv_path}' 파일로 저장되었습니다!")


감정 분석 결과가 '/content/drive/MyDrive/EmoModel/all_reviews_with_emotions.csv' 파일로 저장되었습니다!
